# Portfolio calculaltor

In [6]:
import yfinance as yf
import pandas as pd
from currency_converter import CurrencyConverter
from functions import get_data, intrinsic_value, margin_of_safety, acceptable_buy_price, buy_or_sell

Add the tickers of your portfolio

In [7]:
tickers = ["NOK"] 
fileName = tickers
c = CurrencyConverter()
ROUND_TO_BILLIONS = 0.000001

Create an empty DataFrame to store the information for all tickers

In [8]:
df = pd.DataFrame()

Loop through each ticker and retrieve the information

In [9]:
def process_stock_data(stock_info, wanted_values, rounded_values, convert_to_euros, symbol):
    data = get_data(symbol)
    current_yield = data['Current Yield']
    five_year_growth_rate = data['Growth Rate']
    eps = stock_info.get('trailingEps')
    intrinsic_val = intrinsic_value(eps,current_yield, five_year_growth_rate)
    current_price =stock_info.get('previousClose')
    difference = current_price / intrinsic_value(eps,current_yield, five_year_growth_rate)
    buy_price = round(acceptable_buy_price(intrinsic_val, difference),2)
    safety_margin = margin_of_safety(intrinsic_val, current_price)
    buy_sell = buy_or_sell(buy_price, current_price)
    d = {}
    for k, v in stock_info.items():
        if k in wanted_values:
            if k in rounded_values:
                v *= ROUND_TO_BILLIONS
            elif k in convert_to_euros and stock_info.get('currency') == 'SEK':
                v = round(c.convert(v, 'SEK', 'EUR'),1) * ROUND_TO_BILLIONS
            elif k == 'enterpriseValue':
                v*= ROUND_TO_BILLIONS
            d[k] = [v] 
    #Custom columns
    d['symbol'] = [symbol]
    d['Five year growth'] = [five_year_growth_rate]
    d['Intrinsic value']= [intrinsic_val]
    d['safety margin'] = [safety_margin]
    d['Acceptable buying price'] = [buy_price]
    d['Buy/Sell'] = [buy_sell]
    return d


for symbol in tickers:
    stock = yf.Ticker(ticker=symbol)
    stock_info = stock.get_info()

    # Filter out unwanted values from the dictionary and append to the DataFrame
    wanted_values = ['website', 'longName', 'industry', 'sector', 'fullTimeEmployees', 'auditRisk', 'boardRisk', 'previousClose', 'open', 'dayLow', 'dayHigh', 'dividendRate', 'dividendYield', 'payoutRatio', 'trailingPE', 'forwardPE', 'marketCap', 'priceToSalesTrailing12Months', 'enterpriseValue', 'profitMargins', 'priceToBook', 'trailingEps', 'freeCashflow', 'Growth Estimates']
    # Array for rounded values
    round_values = ['marketCap', 'enterpriseValue', 'freeCashflow']
    # converted values
    convert_to_euros = ['previousClose', 'marketCap', 'open', 'dayLow', 'dayHigh', 'dividendRate', 'freeCashflow'] 

    df_temp = pd.DataFrame.from_dict(data=process_stock_data(stock_info, wanted_values, round_values, convert_to_euros, symbol), orient='columns')
    df = pd.concat([df, df_temp], ignore_index=True) 

Decorate the columns

In [10]:
styles = [
    {'selector': 'th', 'props': [('background-color', 'red'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center')]},
    {'selector': 'tr:nth-child(odd)', 'props': [('background-color', 'pink'), ('text-align', 'center'), ('font-weight', 'bold')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', 'white'), ('color', 'black'), ('text-align', 'center'), ('font-weight', 'bold')]},
]
df_styled = df.style.set_table_styles(styles)

# Display the styled DataFrame
df_styled

,website,industry,sector,fullTimeEmployees,auditRisk,boardRisk,previousClose,open,dayLow,dayHigh,dividendRate,dividendYield,payoutRatio,trailingPE,forwardPE,marketCap,priceToSalesTrailing12Months,enterpriseValue,profitMargins,priceToBook,trailingEps,longName,freeCashflow,symbol,Five year growth,Intrinsic value,safety margin,Acceptable buying price,Buy/Sell
0,https://www.nokia.com,Communication Equipment,Technology,86896,1,1,4.350000,3.970000,3.910000,3.980000,0.120000,0.029000,0.104100,5.168831,8.122449,22114.234368,0.869886,19888.062464,0.169810,1.041612,0.770000,Nokia Oyj,206.000000,NOK,6.720000,5.873542,0.740609,4.350000,Hold


Save the DataFrame to a CSV file with a filename based on the tickers

In [6]:
csv_file = '_'.join(tickers)
df.to_csv(f'{tickers}.csv', index=False)